## KNN Model 57.7% Accuracy

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

from nltk import word_tokenize
from time import time
import pandas as pd
import re
import numpy as np
import gensim
import string
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/ktyser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/ktyser/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
train = pd.read_csv("Train-no-stopwords.csv")
train.sample(5)

,ID,Text,Label
1404,ID_yvwqJbEc,boma lati dongosolo la zachuma lithandza kupez...,ECONOMY
893,ID_hVShUCbO,tambala wameza chimanga kusadabuzika kwa chi...,POLITICS
526,ID_TVpUSKhx,kudzula mpeni wofunikira kwa butchala popha ...,SOCIAL ISSUES
723,ID_bLnPPikX,adzudzula kampani ya enviro consult kamba koch...,SOCIAL
759,ID_cUtBVlcH,ogwira ntchito ku chipatala cha zomba akupitil...,SOCIAL


In [3]:
test = pd.read_csv("Test.csv")
test.sample(5)

,ID,Text
168,ID_NDYxFjoN,Malawi ali pamotoKapito Mkulu wa bungwe loona...
60,ID_EqkeMLWS,"Gogo wa zaka 99 asimbanji? Kwa ambiri, zoti m..."
369,ID_fTVMsJSG,Msungwana Wafa Atangededwa ndi Mwala ku Dansi ...
469,ID_nyAesysa,Mabungwe apereke nzeru pamkangano wa nyanja K...
230,ID_SkRPoSoE,Pulogalamu ya ECRP yatha Pulogalamu yophunzit...


In [4]:
punctuations = string.punctuation + "’¶•@°©®™"

In [5]:
def preprocess_text(text):
    """
    @param text string
    @return text string
    
    This function preprocess a given raw text by normalizing it to lowercase removing the stop words,
    punctuations and lemmatization
    """
        
    #string to lowercase
    txt = text.lower()
    
    # keep only ascii characters
    txt = re.sub(r"[^a-zA-ZÀ-ÿ]", " ", txt)
    
    # punctuation removal and map it to space
    translator = str.maketrans(punctuations, " "*len(punctuations))
    s = txt.translate(translator)
    
    # remove digits 
    no_digits = ''.join([i for i in s if not i.isdigit()])
    cleaner = " ".join(no_digits.split())
    
    # tokenize words and removing stop words 
    word_tokens = word_tokenize(cleaner)
    filtered_sentence = [w for w in word_tokens if len(w)>2 ]
    filtered_sentence = " ".join(filtered_sentence)
    
    return filtered_sentence

In [7]:
train['clean_text'] = train.Text.apply(preprocess_text)
test['clean_text'] = test.Text.apply(preprocess_text)

In [8]:
train.sample(5)

,ID,Text,Label,clean_text
1039,ID_mzUtQMiT,ambwandira msilikali ku salima apolisi chiga...,SOCIAL ISSUES,ambwandira msilikali salima apolisi chigawo ch...
461,ID_RpsAAjdq,kulumana ofunika kulandira chakudya kusankha...,ECONOMY,kulumana ofunika kulandira chakudya kusankha o...
1012,ID_mAsNFIeK,mphekesera zizunza ku mangochi zina ukamva k...,SOCIAL ISSUES,mphekesera zizunza mangochi zina ukamva kamba ...
1279,ID_ujFvdNCX,papa wasankha mamembala atsopano sukulu ya uka...,EDUCATION,papa wasankha mamembala atsopano sukulu ukache...
393,ID_PLSOyJqq,kusowa kwa magazi mankhwala ku qech odwala k...,HEALTH,kusowa kwa magazi mankhwala qech odwala kuchip...


In [11]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

pipeline = Pipeline(
    [
        ("vect", CountVectorizer()),
        ("tfidf", TfidfTransformer()),
        ("classifier", KNeighborsClassifier()),
    ]
)

In [12]:
scores = cross_val_score(pipeline, train['clean_text'], train['Label'], cv=5, scoring='precision_micro')
print(scores)

[0.57291667 0.54006969 0.59930314 0.54703833 0.55749129]


In [13]:
print(f"Final score is {scores.mean()}")

Final score is 0.5633638211382113


In [14]:
pipeline.fit( train['clean_text'], train['Label'])
test.loc[:,"Label"] = pipeline.predict(test['Text'])

In [15]:
test[['ID','Label']]

,ID,Label
0,ID_ADHEtjTi,RELIGION
1,ID_AHfJktdQ,RELIGION
2,ID_AUJIHpZr,RELATIONSHIPS
3,ID_AUKYBbIM,POLITICS
4,ID_AZnsVPEi,EDUCATION
...,...,...
615,ID_zdpOUWyJ,POLITICS
616,ID_zhnOomuu,RELATIONSHIPS
617,ID_zmWHvBJb,LAW/ORDER
618,ID_zphjdFIb,RELATIONSHIPS


In [16]:
#test[['ID','Label']].to_csv("submission.csv", index=False) 

#### Submitted to website and got accuracy of 57.7%